In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('User_actions_1782411_2023_04_27.csv')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62574 entries, 0 to 62573
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   62574 non-null  int64 
 1   order_id  62574 non-null  int64 
 2   action    62574 non-null  object
 3   time      62574 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.9+ MB


In [30]:
df.time = pd.to_datetime(df['time'],format='%d/%m/%y %H:%M')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62574 entries, 0 to 62573
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   user_id   62574 non-null  int64         
 1   order_id  62574 non-null  int64         
 2   action    62574 non-null  object        
 3   time      62574 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 1.9+ MB


In [31]:
created_orders = df[df.action == 'create_order'] \
    .sort_values(['user_id','time'],ascending=[1,2])\
    .copy()

In [32]:
created_orders.head()

,user_id,order_id,action,time
0,1,1,create_order,2022-08-24 01:52:00
4912,1,4683,create_order,2022-08-27 20:56:00
24084,1,22901,create_order,2022-09-02 00:58:00
24343,1,23149,create_order,2022-09-02 02:36:00
1,2,2,create_order,2022-08-24 06:37:00


In [33]:
created_orders['order_numbers'] = created_orders.groupby('user_id').time.rank()

,user_id,order_id,action,time,order_numbers
0,1,1,create_order,2022-08-24 01:52:00,1.0
4912,1,4683,create_order,2022-08-27 20:56:00,2.0
24084,1,22901,create_order,2022-09-02 00:58:00,3.0
24343,1,23149,create_order,2022-09-02 02:36:00,4.0
1,2,2,create_order,2022-08-24 06:37:00,1.0
...,...,...,...,...,...
62560,21399,59583,create_order,2022-09-08 23:56:00,1.5
62564,21399,59587,create_order,2022-09-08 23:58:00,3.0
62563,21400,59586,create_order,2022-09-08 23:57:00,1.0
62566,21401,59589,create_order,2022-09-08 23:58:00,1.0


In [39]:
created_orders['min_order_time'] = created_orders.groupby('user_id').time.transform('min')

In [40]:
created_orders['total_orders'] = created_orders.groupby('user_id').order_id.transform('count')
created_orders.head()

,user_id,order_id,action,time,order_numbers,min_order_time,total_orders
0,1,1,create_order,2022-08-24 01:52:00,1.0,2022-08-24 01:52:00,4
4912,1,4683,create_order,2022-08-27 20:56:00,2.0,2022-08-24 01:52:00,4
24084,1,22901,create_order,2022-09-02 00:58:00,3.0,2022-08-24 01:52:00,4
24343,1,23149,create_order,2022-09-02 02:36:00,4.0,2022-08-24 01:52:00,4
1,2,2,create_order,2022-08-24 06:37:00,1.0,2022-08-24 06:37:00,2


In [43]:
created_orders['date'] = created_orders.time.dt.date

In [44]:
created_orders.head()

,user_id,order_id,action,time,order_numbers,min_order_time,total_orders,date
0,1,1,create_order,2022-08-24 01:52:00,1.0,2022-08-24 01:52:00,4,2022-08-24
4912,1,4683,create_order,2022-08-27 20:56:00,2.0,2022-08-24 01:52:00,4,2022-08-27
24084,1,22901,create_order,2022-09-02 00:58:00,3.0,2022-08-24 01:52:00,4,2022-09-02
24343,1,23149,create_order,2022-09-02 02:36:00,4.0,2022-08-24 01:52:00,4,2022-09-02
1,2,2,create_order,2022-08-24 06:37:00,1.0,2022-08-24 06:37:00,2,2022-08-24


In [52]:
created_orders_by_day = created_orders \
    .groupby(['user_id','date'], as_index=False) \
    .agg({'order_id':'count'})
created_orders_by_day['total_orders'] = created_orders_by_day \
    .groupby(['user_id']).order_id.transform('sum')

In [53]:
created_orders_by_day

,user_id,date,order_id,total_orders
0,1,2022-08-24,1,4
1,1,2022-08-27,1,4
2,1,2022-09-02,2,4
3,2,2022-08-24,1,2
4,2,2022-08-28,1,2
...,...,...,...,...
41521,21398,2022-09-08,1,1
41522,21399,2022-09-08,3,3
41523,21400,2022-09-08,1,1
41524,21401,2022-09-08,1,1


In [54]:
created_orders_by_day['lag'] = created_orders_by_day \
    .groupby(['user_id']).order_id.shift(1)
created_orders_by_day.head()

,user_id,date,order_id,total_orders,lag
0,1,2022-08-24,1,4,NaN
1,1,2022-08-27,1,4,1.0
2,1,2022-09-02,2,4,1.0
3,2,2022-08-24,1,2,NaN
4,2,2022-08-28,1,2,1.0
